# CMB B-mode パワースペクトルからのテンソル-スカラー比 $r$ の推定

## 基礎理論

### モデル

CMBのB-modeパワースペクトル $C_\ell^{BB}$ は、レンズ効果成分とテンソル成分の和として表される：

$$
C_\ell = r \cdot C_\ell^{\rm tens} + C_\ell^{\rm lens}
$$

ここで：
- $r$：テンソル-スカラー比（推定したいパラメータ）
- $C_\ell^{\rm tens}$：$r=1$ のときのテンソルモードパワースペクトル
- $C_\ell^{\rm lens}$：レンズ効果によるB-modeパワースペクトル

### 観測データ

観測されたパワースペクトル $\hat{C}_\ell$ は、真のパワースペクトルにシステマティックエラーを加えたものとして表される：

$$
\hat{C}_\ell = C_\ell^{\rm sys} + C_\ell^{\rm lens}
$$

ここで $C_\ell^{\rm sys}$ は観測されたシステマティック成分。

## 尤度関数

### 対数尤度

テンソル-スカラー比 $r$ の尤度関数は以下で与えられる（PTEP 2023 論文 Sec. 5.3.2 に基づく）：

$$
\ln \mathcal{L}(r) = \sum_{\ell=2}^{\ell_{\rm max}} \left[ -\frac{1}{2}(2\ell+1) \left( \frac{\hat{C}_\ell}{C_\ell(r)} + \ln C_\ell(r) - \frac{2\ell-1}{2\ell+1} \ln \hat{C}_\ell \right) \right]
$$

ここで：
- $\hat{C}_\ell = C_\ell^{\rm sys} + C_\ell^{\rm lens}$（観測値）
- $C_\ell(r) = r \cdot C_\ell^{\rm tens} + C_\ell^{\rm lens}$（モデル）
- $(2\ell+1)$：多重極の縮退度

### 正規化された尤度

数値計算では、オーバーフローを避けるために正規化された尤度を使用：

$$
\mathcal{L}_{\rm norm}(r) = \exp\left[\ln \mathcal{L}(r) - \max_r \ln \mathcal{L}(r)\right]
$$

## 推定アルゴリズム

1. $r$ の初期探索範囲 $[r_{\rm min}, r_{\rm max}]$ を設定
2. グリッドサーチで尤度を最大化する $r$ を探す
3. 最大尤度の周辺に探索範囲を絞り込む
4. イテレーション毎に探索範囲を狭めて精度を向上：$[r_{\rm peak} - r_{\rm peak}/(2(j+1)), r_{\rm peak} + r_{\rm peak}/(2(j+1))]$
5. 収束したら最終的な $\Delta r = r_{\rm estimated}$ を返す

In [ ]:
""" Container for BB power spectra components.
    Stores multipoles ell and the BB spectra for lensing, tensor, and systematics.

    Fields
    ------
        lmax    : int
            Maximum multipole used to define ell = 2:lmax.
        ell     : 1d-array
            Multipole array from 2 to lmax.
        cl_lens : 1d-array
            Lensing BB spectrum (from r=0 fiducial).
        cl_tens : 1d-array
            Tensor BB spectrum (from r=1 fiducial).
        cl_sys  : 1d-array
            Systematic BB spectrum.
"""
struct BBClModel
    lmax::Int
    ell::Vector{Int}
    cl_lens::Vector{Float64}
    cl_tens::Vector{Float64}
    cl_sys::Vector{Float64}
end

""" Construct a BBClModel from fiducial Cℓ and systematics.
    Loads fiducial spectra at r=0 and r=1 via `read_fiducial_cl`, extracts the BB
    component (index 3), and combines them with the provided systematic BB spectrum.

    Parameters
    ----------
        lmax             : int
            Maximum multipole used to define ell = 2:lmax.
        cl_sys           : 1d-array
            Systematic BB power spectrum evaluated on ell = 2:lmax.
        read_fiducial_cl : function
            Function that returns Cℓ arrays for a given tensor-to-scalar ratio r.
            Must accept keyword argument `r` and return an array with BB at index 3.

    Return
    ------
        model: BBClModel
            Struct containing ell, cl_lens, cl_tens, and cl_sys.
"""
function BBClModel(lmax::Int, cl_sys::Vector{Float64}, read_fiducial_cl::Function)


    # Load fiducial power spectra
    cl_r0 = read_fiducial_cl(r=0)
    cl_r1 = read_fiducial_cl(r=1)

    # Extract BB component (index 3 for BB)
    cl_lens = cl_r0[3, :]
    cl_tens = cl_r1[3, :]
    
    ell = collect(2:lmax)
    
    return BBClModel(lmax, ell, cl_lens, cl_tens, cl_sys)
end